<a href="https://colab.research.google.com/github/alessandrossC/ML-for-people/blob/main/%22HW_2_4_%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D0%B8_%D0%B1%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3%D1%83_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням алгоритмів бустингу: XGBoost та LightGBM, а також використаємо бібліотеку HyperOpt для оптимізації гіперпараметрів.

0. Зчитайте дані `train.csv` в змінну `raw_df` та скористайтесь наведеним кодом нижче аби розділити дані на трнувальні та валідаційні і розділити дані на ознаки з матириці Х та цільову змінну. Назви змінних `train_inputs, train_targets, train_inputs, train_targets` можна змінити на ті, які Вам зручно.

  Наведений скрипт - частина отриманого мною скрипта для обробки даних. Ми тут не викнуємо масштабування та обробку категоріальних змінних, бо хочемо це делегувати алгоритмам, які будемо використовувати. Якщо щось не розумієте в наведених скриптах, рекомендую розібратись: навичка читати код - важлива складова роботи в машинному навчанні.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from typing import Tuple, Dict, Any

raw_df = pd.read_csv('/content/train.csv')

# Визначення цільової змінної
target_col = "Exited"

# Визначення вхідних ознак (усі колонки, крім цільової та неінформативних ідентифікаторів)
input_cols = [col for col in raw_df.columns if col not in [target_col, "id", "CustomerId", "Surname"]]

# Визначення категоріальних змінних
categorical_cols = [col for col in input_cols if raw_df[col].dtype == 'object']


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [11]:
# Розділення на тренувальні та валідаційні набори
train_df, val_df = split_train_val(raw_df, target_col)

# Розділення на ознаки та цільову змінну
train_inputs, train_targets = separate_inputs_targets(train_df, input_cols, target_col)
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols, target_col)

1. В тренувальному та валідаційному наборі перетворіть категоріальні ознаки на тип `category`. Можна це зробити двома способами:
 1. `df[col_name].astype('category')`, як було продемонстровано в лекції
 2. використовуючи метод `pd.Categorical(df[col_name])`

In [12]:
# Перетворення категоріальних ознак на тип category (перший підхід)
for col in categorical_cols:
    train_inputs[col] = train_inputs[col].astype('category')

# Перетворення категоріальних ознак на тип category (другий підхід)
for col in categorical_cols:
    val_inputs[col] = pd.Categorical(val_inputs[col])

# Виведення розміру отриманих наборів
print(train_inputs.shape, train_targets.shape, val_inputs.shape, val_targets.shape)

(12000, 10) (12000,) (3000, 10) (3000,)


2. Навчіть на отриманих даних модель `XGBoostClassifier`. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів XGBoostClassifier - тут https://xgboost.readthedocs.io/en/stable/parameter.html#global-config

  **Важливо:** зробіть такі налаштування `XGBoostClassifier` аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Можна також, якщо працюєте в Google Colab, увімкнути можливість використання GPU (`Runtime -> Change runtime type -> T4 GPU`) і встановити параметр `device='cuda'` в `XGBoostClassifier` для пришвидшення тренування бустинг моделі.
  
  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням DecisionTrees раніше. Чи вийшло покращити якість?

In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Ініціалізація моделі XGBoostClassifier
xgb_model = XGBClassifier(
    use_label_encoder=False,  # Вимикає попередження про кодування міток
    eval_metric='logloss',  # Вибираємо функцію втрат логарифмічну
    missing=np.nan,  # Обробка пропущених значень
    tree_method='hist',  # Оптимізація швидкості
    enable_categorical=True,  # Автоматична обробка категоріальних змінних
    random_state=42
)

# Навчання моделі
xgb_model.fit(train_inputs, train_targets)

# Прогнози ймовірностей для оцінки AUROC
train_preds = xgb_model.predict_proba(train_inputs)[:, 1]
val_preds = xgb_model.predict_proba(val_inputs)[:, 1]

# Обчислення AUROC
train_auc = roc_auc_score(train_targets, train_preds)
val_auc = roc_auc_score(val_targets, val_preds)

print(f"AUROC на тренувальній вибірці: {train_auc}")
print(f"AUROC на валідаційній вибірці: {val_auc}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:55:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


AUROC на тренувальній вибірці: 0.9917909049264981
AUROC на валідаційній вибірці: 0.921015844708142


Так, модель показує дуже хороші результати. Значення AUROC > 0.9 вказує на високу якість класифікації.

Різниця між AUROC на тренуванні (0.9918) і валідації (0.9210) вказує на певне перенавчання (overfitting)

XGBoost AUROC: 0.921
DecisionTree AUROC: 0.915

XGBoost може бути перенавченим, через що не дає значного приросту на валідації

3. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `XGBoostClassifier` з лекції знайдіть оптимальні значення гіперпараметрів `XGBoostClassifier` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **20**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. В ній ми маємо задати loss - це може будь-яка метрика, але бажано використовувтаи ту, яка цільова в вашій задачі. Чим менший лосс - тим ліпша модель на думку hyperopt. Тож, тут нам треба задати loss - негативне значення AUROC. В лекції ми натомість використовували Accuracy.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_clf` модель `XGBoostClassifier` з найкращими гіперпараметрами
    - навчіть модель `final_clf`
    - оцініть якість моделі `final_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (2) цього завдання?

In [16]:
!pip install hyperopt

In [15]:
from hyperopt import fmin, tpe, hp, Trials
from xgboost import XGBClassifier

# Функція для оптимізації
def objective(params):
    model = XGBClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha'],
        missing=np.nan,
        tree_method='hist',
        enable_categorical=True,
        random_state=42
    )

    model.fit(train_inputs, train_targets)
    val_preds = model.predict_proba(val_inputs)[:, 1]
    val_auc = roc_auc_score(val_targets, val_preds)

    return -val_auc  # Hyperopt мінімізує значення, тому беремо негативний AUROC

# Простір пошуку гіперпараметрів
param_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0, 10)
}

# Запуск Hyperopt для пошуку оптимальних параметрів
trials = Trials()
best_params = fmin(fn=objective, space=param_space, algo=tpe.suggest, max_evals=20, trials=trials)

# Конвертація параметрів у правильний формат
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])

# Виведення найкращих параметрів
print("Найкращі гіперпараметри:")
best_params

100%|██████████| 20/20 [00:05<00:00,  3.43trial/s, best loss: -0.9368996501817684]
Найкращі гіперпараметри:


{'colsample_bytree': 0.662321366340816,
 'learning_rate': 0.12151493914778948,
 'max_depth': 3,
 'n_estimators': 120,
 'reg_alpha': 3.6164607204061614,
 'reg_lambda': 5.252743948749027,
 'subsample': 0.7645380854254465}

In [17]:
# Ініціалізація моделі XGBoostClassifier з найкращими параметрами
final_clf = XGBClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    reg_lambda=best_params['reg_lambda'],
    reg_alpha=best_params['reg_alpha'],
    missing=np.nan,
    tree_method='hist',
    enable_categorical=True,
    random_state=42
)

# Навчання моделі
final_clf.fit(train_inputs, train_targets)

# Прогнози ймовірностей для оцінки AUROC
train_preds_final = final_clf.predict_proba(train_inputs)[:, 1]
val_preds_final = final_clf.predict_proba(val_inputs)[:, 1]

# Обчислення AUROC
train_auc_final = roc_auc_score(train_targets, train_preds_final)
val_auc_final = roc_auc_score(val_targets, val_preds_final)

# Виведення результатів
print(f"AUROC на тренувальній вибірці: {train_auc_final}")
print(f"AUROC на валідаційній вибірці: {val_auc_final}")

AUROC на тренувальній вибірці: 0.9426115895042448
AUROC на валідаційній вибірці: 0.9368996501817684


Менше перенавчання – різниця між Train і Validation AUROC стала меншою.
Validation AUROC підвищився з 0.9210 до 0.9369.
Модель стала більш узагальненою, тому що її продуктивність на валідації ближча до тренувальної.

4. Навчіть на наших даних модель LightGBM. Параметри алгоритму встановіть на свій розсуд, ми далі будемо їх тюнити. Рекомендую тренувати не дуже складну модель.

  Опис всіх конфігураційних параметрів LightGBM - тут https://lightgbm.readthedocs.io/en/latest/Parameters.html

  **Важливо:** зробіть такі налаштування LightGBM аби він самостійно обробляв незаповнені значення в даних і обробляв категоріальні колонки.

  Аби передати категоріальні колонки в LightGBM - необхідно виявити їх індекси і передати в параметрі `cat_feature=cat_feature_indexes`

  Після тренування моделі
  1. Виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах.
  2. Зробіть висновок про отриману модель: вона хороша/погана, чи є high bias/high variance?
  3. Порівняйте якість цієї моделі з тою, що ви отрмали з використанням XGBoostClassifier раніше. Чи вийшло покращити якість?

5. Використовуючи бібліотеку `Hyperopt` і приклад пошуку гіперпараметрів для `LightGBM` з лекції знайдіть оптимальні значення гіперпараметрів `LightGBM` для нашої задачі. Задайте свою сітку гіперпараметрів виходячи з тих параметрів, які ви б хотіли перебрати. Поставте кількість раундів в підборі гіперпараметрів рівну **10**.

  **Увага!** Для того, аби скористатись hyperopt, нам треба задати функцію `objective`. І тут ми також ставимо loss - негативне значення AUROC, як і при пошуці гіперпараметрів для XGBoost. До речі, можна спробувати написати код так, аби в objective передавати лише модель і не писати схожий код двічі :)

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення гіперпараметрів
    - створіть в окремій зміній `final_lgb_clf` модель `LightGBM` з найкращими гіперпараметрами
    - навчіть модель `final_lgb_clf`
    - оцініть якість моделі `final_lgb_clf` на тренувальній і валідаційній вибірках з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пунктом (4) цього завдання?

In [18]:
!pip install lightgbm

In [19]:
%%bash
sudo apt-get update
sudo apt-get install -y build-essential cmake git wget unzip
sudo apt-get install -y libboost-dev libboost-system-dev libboost-filesystem-dev
sudo apt-get install -y libboost-iostreams-dev libboost-program-options-dev libboost-regex-dev
sudo apt-get install -y libboost-thread-dev libboost-chrono-dev libboost-date-time-dev
sudo apt-get install -y libboost-atomic-dev libboost-serialization-dev
sudo apt-get install -y python3-pip

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,661 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/u

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [20]:
%%bash
sudo apt-get install -y ocl-icd-libopencl1 clinfo
sudo apt-get install -y nvidia-opencl-dev opencl-headers

Reading package lists...
Building dependency tree...
Reading state information...
clinfo is already the newest version (3.0.21.02.21-1).
ocl-icd-libopencl1 is already the newest version (2.2.14-3).
ocl-icd-libopencl1 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Reading package lists...
Building dependency tree...
Reading state information...
nvidia-opencl-dev is already the newest version (11.5.1-1ubuntu1).
The following NEW packages will be installed:
  opencl-headers
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 1,754 B of archives.
After this operation, 12.3 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 opencl-headers all 3.0~2022.01.04-1 [1,754 B]
Fetched 1,754 B in 0s (4,927 B/s)
Selecting previously unselected package opencl-headers.
(Reading database ... 125788 files and directories currently installed.)
Preparing to unpack .../opencl-headers_3.0~20

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [21]:
%%bash
git clone --recursive https://github.com/microsoft/LightGBM
cd LightGBM
mkdir build
cd build
cmake -DUSE_CUDAP=1 ..
make -j4

Submodule path 'external_libs/compute': checked out '36350b7de849300bd3d72a05d8bf890ca405a014'
Submodule path 'external_libs/eigen': checked out '3147391d946bb4b6c68edd901f2add6ac1f31f8c'
Submodule path 'external_libs/fast_double_parser': checked out '252029ddac664370bdda3f0761675785d92a1573'
Submodule path 'external_libs/fast_double_parser/benchmarks/dependencies/abseil-cpp': checked out 'd936052d32a5b7ca08b0199a6724724aea432309'
Submodule path 'external_libs/fast_double_parser/benchmarks/dependencies/double-conversion': checked out 'f4cb2384efa55dee0e6652f8674b05763441ab09'
Submodule path 'external_libs/fmt': checked out '8303d140a1a11f19b982a9f664bbe59a1ccda3f4'
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI i

Cloning into 'LightGBM'...
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
Cloning into '/content/LightGBM/external_libs/eigen'...
Cloning into '/content/LightGBM/external_libs/fast_double_parser'...
Cloning into '/content/LightGBM/external_libs/fmt'...
Submodule 'benchmark/dependencies/abseil-cpp' (https://github.com/abseil/abseil-cpp.git) registered for path 'external_libs/fast_double_parser/benchmarks/dependencies/abseil-cpp'
Submodule 'benchmark/dependencies/double-conversion' (https://github.com/google/doub

In [23]:
import lightgbm as lgb
print(lgb.__version__)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


4.5.0


In [32]:
cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in categorical_cols]

In [33]:
import lightgbm as lgb
from sklearn.metrics import classification_report

# Перетворення категоріальних ознак у формат category
for col in categorical_cols:
    train_inputs[col] = train_inputs[col].astype('category')
    val_inputs[col] = val_inputs[col].astype('category')

# Отримання індексів категоріальних ознак
cat_feature_indexes = [train_inputs.columns.get_loc(col) for col in categorical_cols]

# Ініціалізація моделі LightGBM
lgb_clf = lgb.LGBMClassifier(
    max_depth=3,
    n_estimators=50,
    learning_rate=0.1,
    categorical_feature=cat_feature_indexes,  # для автоматичного розпізнавання категорійних ознак
    missing=np.nan,  # явне вказування пропущених значень
    # device='cuda'  # використовувати GPU для прискорення обчислень
)

# Навчання моделі
lgb_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)])

# Прогнозування
train_pred = lgb_clf.predict(train_inputs)
val_pred = lgb_clf.predict(val_inputs)

# Оцінка результатів
print(classification_report(train_targets, train_pred, digits=4))
print(classification_report(val_targets, val_pred, digits=4))

[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Warning] categorical_feature is set=1,2, categorical_column=1,2 will be ignored. Current value: categorical_feature=1,2
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001099 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 843
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 10
[LightGBM] [Warning] Unknown parameter: missing
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(


In [44]:
# Функція для оптимізації гіперпараметрів
def objective(params):
    clf = lgb.LGBMClassifier(
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        num_leaves=int(params['num_leaves']),
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        min_split_gain=params['min_split_gain'],
        categorical_feature=cat_feature_indexes,  # Індекси категорійних ознак
        missing=np.nan,
        verbose=-1  # Отключение вывода логов модели
    )

    clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)], eval_metric="logloss")
    pred = clf.predict(val_inputs)
    accuracy = accuracy_score(val_targets, pred)

    return {'loss': -accuracy, 'status': STATUS_OK}

# Простір гіперпараметрів
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 25),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'reg_alpha': hp.uniform('reg_alpha', 0, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'min_split_gain': hp.uniform('min_split_gain', 0, 0.1)
}

# Оптимізація
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

# Перетворення значень гіперпараметрів у кінцеві типи
best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = int(best['max_depth'])
best['num_leaves'] = int(best['num_leaves'])
best['min_child_weight'] = int(best['min_child_weight'])

print("Найкращі гіперпараметри: ", best)

# Навчання фінальної моделі з найкращими гіперпараметрами
final_clf = lgb.LGBMClassifier(
    n_estimators=best['n_estimators'],
    learning_rate=best['learning_rate'],
    max_depth=best['max_depth'],
    num_leaves=best['num_leaves'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    reg_alpha=best['reg_alpha'],
    reg_lambda=best['reg_lambda'],
    min_split_gain=best['min_split_gain'],
    categorical_feature=cat_feature_indexes,
    missing=np.nan,
    verbose=-1  # Отключение вывода логов модели
)

# Обучение модели
final_clf.fit(train_inputs, train_targets, eval_set=[(val_inputs, val_targets)], eval_metric="logloss")

# Предсказание классов
final_pred = final_clf.predict(val_inputs)
final_accuracy = accuracy_score(val_targets, final_pred)

# Предсказание вероятностей для AUROC
train_probs = final_clf.predict_proba(train_inputs)[:, 1]
val_probs = final_clf.predict_proba(val_inputs)[:, 1]

# Вычисление AUROC
train_auc = roc_auc_score(train_targets, train_probs)
val_auc = roc_auc_score(val_targets, val_probs)

# Вывод результатов
print("AUROC на тренувальній вибірці: {:.4f}".format(train_auc))
print("AUROC на валідаційній вибірці: {:.4f}".format(val_auc))
print("Точність на валідаційній вибірці: {:.4f}".format(final_accuracy))
print("\nКласифікаційний звіт:")
print(classification_report(val_targets, final_pred, digits=4))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(



 10%|█         | 1/10 [00:00<00:06,  1.43trial/s, best loss: -0.89]

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(



 20%|██        | 2/10 [00:02<00:10,  1.26s/trial, best loss: -0.8916666666666667]

...

 90%|█████████ | 9/10 [00:07<00:00,  1.28trial/s, best loss: -0.902]

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(

/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(



100%|██████████| 10/10 [00:07<00:00,  1.27trial/s, best loss: -0.902]
Найкращі гіперпараметри:  {'colsample_bytree': 0.6158681877045149, 'learning_rate': 0.19022016814568107, 'max_depth': 3, 'min_child_weight': 9, 'min_split_gain': 0.0931354284327202, 'n_estimators': 225, 'num_leaves': 47, 'reg_alpha': 0.16344002088096765, 'reg_lambda': 0.16844927690919687, 'subsample': 0.884833947889754}


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(


AUROC на тренувальній вибірці: 0.9546
AUROC на валідаційній вибірці: 0.9345
Точність на валідаційній вибірці: 0.9020

Класифікаційний звіт:
              precision    recall  f1-score   support

         0.0     0.9219    0.9582    0.9397      2390
         1.0     0.8062    0.6820    0.7389       610

    accuracy                         0.9020      3000
   macro avg     0.8641    0.8201    0.8393      3000
weighted avg     0.8984    0.9020    0.8989      3000



Оптимізація дала незначне покращення у точності та AUROC на тренувальних даних.
UROC на валідаційних даних майже не змінилася (мінус 0.24%), що означає, що модель не перенавчилася.

Загалом модель стала трохи краще, але ефект оптимізації гіперпараметрів не дуже великий.

6. Оберіть модель з експериментів в цьому ДЗ і зробіть новий `submission` на Kaggle та додайте код для цього і скріншот скора на публічному лідерборді.
  
  **Напишіть коментар, чому ви обрали саме цю модель?**

  І я вас вітаю - це останнє завдання з цим набором даних 💪 На цьому етапі корисно проаналізувати, які моделі показали себе найкраще і подумати, чому.

In [51]:
import pandas as pd

# 🔹 Завантаження тестових даних
test_data_path = "/content/test.csv"
test_df = pd.read_csv(test_data_path)

# 🔹 Перевіряємо, чи є 'id' у test_df
if 'id' not in test_df.columns:
    raise ValueError("Колонка 'id' відсутня у тестових даних!")

# 🔹 Видаляємо непотрібні колонки (як у тренувальних даних)
input_cols = [col for col in train_inputs.columns]  # Використовуємо ті самі ознаки, що і в train_inputs
test_inputs = test_df[input_cols].copy()

# 🔹 Гарантія, що категоріальні колонки мають такий самий тип (category)
categorical_cols = train_inputs.select_dtypes(include=['category']).columns.tolist()

for col in categorical_cols:
    test_inputs[col] = test_inputs[col].astype('category')

    # 🔹 Переконуємося, що всі категорії з train є в test (додаємо, якщо треба)
    test_inputs[col] = test_inputs[col].cat.set_categories(train_inputs[col].cat.categories)

# 🔹 **Гарантія, що стовпці test_inputs та train_inputs співпадають**
missing_cols = set(train_inputs.columns) - set(test_inputs.columns)

# Додаємо відсутні колонки з нулями
for col in missing_cols:
    test_inputs[col] = 0

# 🔹 Упорядковуємо стовпці так само, як у train_inputs
test_inputs = test_inputs[train_inputs.columns]

# 🔹 Передбачення ймовірностей для тестового набору
test_preds = final_clf.predict_proba(test_inputs)[:, 1]  # Ймовірність класу 1 (Exited)

# 🔹 Створення `submission.csv`
submission = pd.DataFrame({
    'id': test_df['id'],  # Використовуємо реальний ID з test_df
    'Exited': test_preds
})

# 🔹 Збереження у файл CSV
submission_path = "/content/submission.csv"
submission.to_csv(submission_path, index=False)

print(f"✅ Файл submission.csv успішно збережено за шляхом: {submission_path}")

# 🔹 Виведення перших рядків `submission.csv`
submission.head()


✅ Файл submission.csv успішно збережено за шляхом: /content/submission.csv


,id,Exited
0,15000,0.049632
1,15001,0.008179
2,15002,0.053421
3,15003,0.526616
4,15004,0.031371
